In [5]:
import pandas as pd
import numpy as np
import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import tensorflow as tf

# GPU 사용 설정
device_name = tf.test.gpu_device_name()
if not tf.config.list_physical_devices('GPU'):
    print("GPU를 찾을 수 없습니다. 런타임 유형을 GPU로 변경해주세요.")
else:
    print(f"GPU 장치: {device_name}")

GPU 장치: /device:GPU:0


2023-09-20 04:53:04.860005: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-09-20 04:53:04.860026: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-09-20 04:53:04.860033: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-09-20 04:53:04.860073: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-20 04:53:04.860091: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# 1. 수기 라벨링한 속성 분류
- 처리 프로세스
    - 추출된 총 7000개 정도의 속성에서 출현 횟수 5회 이상인 속성을 기준 총 1,076개의 속성에 대해 수기 라벨링 진행
    - fasttext로 유사 단어를 추출할 때 오분류되기 쉬운 단어(ex: quality, perform)도 추가로 수기 라벨링 진행
- 분류 기준
    - battery : 배터리에 관련된 속성
    - body : 스마트폰의 외관과 내구성, 디자인적 요소에 관련된 속성
    - display : 화면 자체에 대한 설명과 성능에 관련된 속성
    - camera : 카메라 성능과 관련 기능에 관련된 속성
    - function : 스마트폰에 탑재된 기능에 관련된 속성
    - accessory : 스마트폰과 동봉되는 액세서리에 관련된 속성
    - mainboard (processor + storage) : 처리 성능과 저장 용량에 관련된 속성  
    => 수기 라벨링을 통해 정의된 스마트폰 속성 개수 : 421개  
    => 총 출현 횟수 기준으로 약 90% 정도의 속성을 살펴봄


In [6]:
lst_batt = ['charg', 'recharg', 'batteri_charg', 'batteri', 'batter', 'bateri', 'battari', 
            'baterri', 'phone_batteri', 'iphon_batteri', 'intern_batteri', 'batteri_backup', 
            'batteri_replac', 'replac_batteri', 'standbi_time', 'batteri_percentag', 'batteri_usag', 
            'batteri_life', 'batter_life', 'batteri_live', 'bateri_life', 'batteri_health', 'batteri_capac', 
            'batteri_perform', 'batteri_power', 'batteri_qualiti', 'batteri_condit', 'batteri_time', 
            'charg_time', 'fast_charg']


lst_cam = ['camera', 'camara', 'cam', 'front_camera', 'front_face_camera', 'back_camera', 'rear_camera', 
           'rear_face_camera', 'selfi_camera', 'face_camera', 'main_camera', 'photo', 'imag', 
           'pixel', 'camera_qualiti', 'pictur_qualiti', 'photo_qualiti', 'imag_qualiti', 'video_qualiti', 
           'shutter_sound', 'camera_shutter', 'shutter', 'camera_shutter_sound', 'camera_sound', 'shutter_nois', 
           'camera_nois', 'camera_len', 'camera_lens', 'len', 'lens', 'camera_glass', 'camera_featur', 'lens_cover' 
           'camera_function', 'camera_app']
# pic, pictur, video

lst_dis = ['of_the_screen', 'screen', 'phone_screen', 'display', 'amol_display', 'display_screen', 'screen_display', 
           'retina_display', 'glass_screen', 'touch_screen', 'touchscreen', 'lcd', 'lcd_screen', 'led_screen', 
           'hd_screen', 'screen_size', 'size_of_the_screen',  'graphic', 'screen_color', 'screen_qualiti', 
           'screen_resolut', 'resolut', 'bright', 'screen_bright', 'crack_screen', 'screen_burn', 
           'screen_touch', 'sensit', 'screen_sensit', 'gorilla_glass', 'temper_glass', 'panel', 
           
          'size_screen'] #### 추가한거

lst_body = ['cosmet_condit', 'hardwar', 'bodi', 'bezel', 'form_factor', 'plate', 'visual', 'phone_look', 
            'look_phone', 'exterior', 'cosmet', 'design', 'style', 'aesthet', 'shape', 'back_panel', 
            'size', 'size_of', 'size_of_the_phone', 'weight', 'grip', 'width',  'back_of_the_phone', 
            'paint', 'gold', 'silver', 'edg', 'surfac', 'frame', 'materi', 'plastic', 'metal', 
            'metal_frame', 'glass',  'aluminum', 'back_glass', 'glass_back', 'glass_front', 'glass_cover', 
            'port', 'charg_port', 'charger_port', 'usb_port', 'headphon_port', 'lightn_port', 'sim_card_port', 
            'audio_port', 'speaker_grill', 'speaker_hole', 'button', 'home_button', 'power_button', 
            'volum_up_button', 'volum_button', 'lock_button', 'back_button', 'mute_button', 'silent_button', 
            'volum_down_button', 'mute_switch', 'menu_button', 'botton', 'home_key', 
            'build', 'built', 'construct', 'chassi', 'durabl', 'volum_key', 'power_key', 'scratch', 'crack', 
            'waterproof', 'water_resist', 'water_proof', 'water_damag', 'waterproof_featur', 'built_qualiti', 
            'plastic_cover', 'metal_cover', 'on_off_button', 
           
           'size_phone', 'back_phone'] #### 추가한거
# outlet, look, corner

lst_func = ['flashlight', 'led_light', 'mute_switch', 'speaker', 'phone_speaker', 'ear_speaker', 
            'extern_speaker', 'earpiec_speaker', 'intern_speaker', 'mic', 'earpiec', 'ear_piec', 
            'true_tone', 'sound_system', '3d_touch',  'scanner', 'gyroscop', 'sound', 'call_volum', 
            'audio', 'microphon', 'speaker_phone', 'speakerphon', 'sound_qualiti', 'speaker_qualiti', 
            'audio_qualiti', 'call_qualiti', 'function', 'phone_function', 'touch_function', 
            'fingerprint_reader', 'fingerprint_scanner', 'fingerprint', 'fingerprint_sensor', 
            'finger_print_scanner', 'fingerprint_recognit', 'face_recognit', 'facial_recognit', 
            'face_id', 'faceid', 'touch', 'touch_id', 'touchid', 'sensor', 'proxim_sensor', 'vibrat',  
            'voic_command', 'voic_control', 'air_gestur', 'gestur', 'recognit', 'volum_control', 
            'bluetooth', 'blue_tooth', 'blutooth', 'ringer', 'airplan_mode', 'screen_rotat', 'nfc', 
            'secur', 'secur_featur', 'privaci', 'wireless_charg']

lst_main = ['motherboard', 'mother_board', 'logic_board', 'circuit_board', 'main_board', 'baseband', 
            'circuit', 'board']

lst_proc = ['processor', 'ram', 'cpu', 'quad_core', 'snapdragon_processor', 'ap', 'exyno_processor', 
            'core', 'process_speed', 'processor_speed', 'process_power']
# 'heat', 'heat_up'

lst_stor = ['storag', 'intern_storag', 'storag_space', 'memori_space', ' expand_storag', 'space', 'gb', 
            '16gb', '32gb', '8gb', '16_gb', 'memori', 'intern_memori', 'expand_memori', 'memori_card', 
            'memori_chip', 'extern_memori', 'rom', 'storag_capac', 'memori_capac']

lst_acc = ['accessori', 'accesori', 'charger', 'wall_charger', 'phone_charger', 'batteri_charger', 
           'wireless_charger', 'fast_charger', 'usb_charger', 'car_charger', 'charg_block', 'power_brick',
           'charg_brick', 'charger_wire', 'charg_wire', 'wire', 'earphon', 'ear_phone',  'batteri_charg', 
           'headset', 'handset', 'head_phone', 'headphon', 'ear_bud', 'earbud', 'earpod', 'dock', 'cabl',
           'charg_cabl', 'charger_cabl', 'usb_cabl', 'lightn_cabl', 'power_cabl', 'light_cabl', 'lighten_cabl', 
           'data_cabl', 'jack', 'headphon_jack', 'earphon_jack', 'audio_jack', 'phone_jack', 'adapt', 
           'adaptor', 'power_adapt', 'wall_adapt', 'charger_adapt', 'charg_adapt', 'headphon_adapt', 'plug', 
           'charg_plug', 'wall_plug', 'charger_plug', 'ear_plug', 'power_plug', 'cord', 'charg_cord',
           'power_cord', 'usb_cord', 'charger_cord', 'connector', ' lightn_connector', 'usb_connector']

In [7]:
plus_body = ['build_qualiti', 'built_qualiti', 'qualiti_materi', 'qualiti_build', 'cosmet_qualiti', 
             'aluminum_build_qualiti', 'design_qualiti','materi_qualiti', '_resist_qualiti', 
             'construct_qualiti', 'perform_camera_qualiti_waterproof', 
            
             'condit_perform', 'perform_style', 'perform_design', 'style_perform']

plus_cam = ['pic_qualiti', 'qualiti_of_the_pictur', 'qualiti_of_photo', 'qualiti_pictur', 'qualiti_of_the_photograph', 
            'qualiti_of_the_camera', 'camera_pictur_qualiti', 'qualiti_in_pictur', 'qualiti_of_imag', 'len_qualiti', 
            'qualiti_of_pic', 'indoor_photo_qualiti', 'shot_qualiti', 'qualiti_imag', 'qualiti_photo', 
            'qualiti_of_camera', 'pictur_take_qualiti', 'qualiti_pic', 'camara_qualiti', 
            'perform_camera_qualiti_waterproof', 'oper_camera_qualiti', 'pictut_qualiti', 
            
            'camera_perform']


plus_func = ['voic_qualiti', 'qualiti_sound', 'qualiti_speaker', 'qualiti_of_the_sound', 'voic_call_qualiti', 
            'qualiti_of_sound', 'microphon_qualiti', 'qualiti_of_speaker', 'qualiti_of_the_audio', 
            'call_qualiti_mic', 'sound_qualiti_batteri_life', 'screen_resolut_speakerphon_qualiti', 
            'loud_speaker_qualiti', 'video_sound_qualiti', 'call_qualiti_featur', 'soudn_qualiti', 
            'phone_call_qualiti', 'call_qualiti_bluetooth_qualiti', 'signal_qualiti', 
            
            'bluetooth_perform', 'audio_perform']

plus_dis = ['graphic_qualiti', 'hd_qualiti', 'screen_resolut_speakerphon_qualiti', 'qualiti_of_screen', 
            'size_screen_video_qualiti', 'qualiti_of_the_screen', 'display_batteri_build_qualiti', 
            
            'screen_perform', 'graphic_perform', 'touch_screen_perform']

plus_proc = ['process_speed_qualiti_build',
             'processor_perform', 'speed_perform']

plus_batt = ['batter_qualiti', 'sound_qualiti_batteri_life', 'display_batteri_build_qualiti', 
            
             'batter_perform', 'batt_perform']

plus_acc = ['headset_qualiti']

In [8]:
lst_other = ['back_cover', 'cover', 'coverag', 'charg_port_cover', 
         'screen_cover', 'glass_cover', 'port_cover',
       'batteri_cover', 'usb_port_cover', 'usb_cover', 'servic_coverag',
       'recoveri', 'charg_cover', 'custom_recoveri', 'case_cover',
       'speaker_cover', 'coverag_area', 'phone_cover', 'charger_cover',
       'flip_cover', 'glass_protect_cover', 'cover_color', 'bodi_cover',
       'cover_charger', 'metal_cover', 'lay_on_plastic_screen_cover',
       'simcard_cover', 'oleophob_cover', 'sim_card_cover', 'ipnon_cover',
       'coverif', 'face_cover', 'glass_screen_cover',
       'protect_temper_glass_cover', 'cover_beack', 'plastic_back_cover',
       'batteri_charg_cover', 'cover_flap', 'usb_port_back_cover',
       'cover_to', 'plug_cover', 'sim_cover', 'cell_coverag',
       'back_batteri_cover', 'back_door_cover', 'flip_cover_folio_case',
       'plastic_batteri_cover', 'covers', 'cell_coverga', 'plug_in_cover',
       'coverag_bar', 'glass_len_cover', 'len_cover', 'front_glass_cover',
       'data_coverag', 'hard_cover', 'camera_cover', 'lens_cover',
       'charger_port_cover', 'recoveri_mode', 'back_cover_protector',
       'warranti_coverag', 'dust_cover', 'batteri_back_cover',
            
        'slot', 'sim_card_slot', 'sd_card_slot', 'sim_slot', 'sd_slot', 'memori_card_slot', 
        'sim_tray', 'tray', 'sim_card_tray', 'sim_card_holder', 'sim_card_reader',    
        'screen_protector', 'glass_screen_protector', 'protector', 'screen_protect', 
        'temper_glass_screen_protector', 'protect', 'film', 'screen_cover', 
        'glass_protect_screen', 'plastic_screen_protector', 'protect_screen']

In [9]:
lst_batt.extend(plus_batt)
lst_body.extend(plus_body)
lst_cam.extend(plus_cam)
lst_func.extend(plus_func)
lst_dis.extend(plus_dis)
lst_proc.extend(plus_proc)
lst_acc.extend(plus_acc)

In [10]:
hand_batt = list(set(lst_batt))
hand_body = list(set(lst_body))
hand_dis = list(set(lst_dis))
hand_cam = list(set(lst_cam))
hand_func = list(set(lst_func))
hand_proc = list(set(lst_proc))
hand_stor = list(set(lst_stor))
hand_acc = list(set(lst_acc))
hand_main = list(set(lst_main))
hand_other = list(set(lst_other))

In [16]:
print('battery:', len(hand_batt))
print('body:', len(hand_body))
print('display:', len(hand_dis))
print('camera:', len(hand_cam))
print('function:', len(hand_func))
print('processor:', len(hand_proc))
print('storage:', len(hand_stor))
print('accessory:', len(hand_acc))
print('mainboard:', len(hand_main))
print('')
print('수기로 라벨링한 속성의 개수:', len(hand_batt)+ len(hand_body)+
      len(hand_dis)+ len(hand_cam)+ len(hand_func)+ len(hand_proc)+ 
      len(hand_stor)+ len(hand_acc)+ len(hand_main))

battery: 35
body: 97
display: 43
camera: 57
function: 84
processor: 14
storage: 20
accessory: 63
mainboard: 8

수기로 라벨링한 속성의 개수: 421


In [12]:
dic_final = {
    'bat': lst_batt,
    'body': lst_body,
    'dis': lst_dis,
    'cam': lst_cam,
    'main': lst_main,
    'proc': lst_proc,
    'stor': lst_stor,
    'func': lst_func,
    'acc': lst_acc, 
    'other':lst_other
}

In [86]:
df_dic = pd.read_csv('추출속성_데이터프레임.csv')
df_dic = df_dic[df_dic['aspect']==df_dic['aspect']]
df_dic.head()

,aspect,ori_aspect,pre_aspect,count,all_cnt
0,phone,phone,phone,10667,10850
1,battery,battery,batteri,9048,9255
2,screen,screen,screen,5880,5956
3,works,works,work,5843,8067
4,iphone,iphone,iphon,4835,5024


In [87]:
# 수기 라벨링한 속성 분류
def sort_aspect(aspect):
    found_in_list = np.nan
    for list_name, my_list in dic_final.items():
        if aspect in my_list:
            found_in_list = list_name
    return found_in_list

In [88]:
df_dic['feature'] = np.nan
df_dic['feature'] = df_dic['pre_aspect'].apply(sort_aspect)

In [89]:
dic_final = {}

df_only = df_dic[df_dic['feature']=='bat']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['bat'] = lst_combine

df_only = df_dic[df_dic['feature']=='body']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['body'] = lst_combine

df_only = df_dic[df_dic['feature']=='dis']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['dis'] = lst_combine

df_only = df_dic[df_dic['feature']=='cam']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['cam'] = lst_combine

df_only = df_dic[df_dic['feature']=='main']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['main'] = lst_combine

df_only = df_dic[df_dic['feature']=='proc']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['proc'] = lst_combine

df_only = df_dic[df_dic['feature']=='stor']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['stor'] = lst_combine

df_only = df_dic[df_dic['feature']=='func']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['func'] = lst_combine

df_only = df_dic[df_dic['feature']=='acc']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['acc'] = lst_combine

df_only = df_dic[df_dic['feature']=='other']
lst_combine = []
for index, row in df_only.iterrows():
    lst_combine.extend([row['aspect'], row['ori_aspect'], row['pre_aspect']])
lst_combine = list(set(lst_combine))
dic_final['other'] = lst_combine

In [90]:
reverse_dic_final = {value: key for key, values in dic_final.items() for value in values}

# 2. fasttext로 유사 속성 추가
- fasttext : 단어 임베딩 모델
    - 텍스트 데이터를 기반으로 단어들을 벡터로 표현하여 유사한 단어를 찾는 등 다양한 자연어 처리 작업에 사용될 수 있는 모델  
- 사용 목적 
    - 다수 출현된 속성 이외에도 단순 오타로 인해 분류되지 않거나 의미가 유사한 속성들을 추가로 분류하고자 사용
- 처리 프로세스
    1. 기존의 토큰화된 문장에서 합성어의 띄어쓰기를 '_'로 대체하여 토큰을 재구성
    2. 수기 라벨링을 통해 분류되지 않은 (출현 횟수 5회 미만) 속서에 대해 fasttext로 유사도가 가장 높은 단어 추출  
    3. 유사도가 가장 높은 단어가 이미 배정된 주요 속성이 있을 경우 해당 주요 속성으로 배정   
        a. 유사도가 가장 높은 단어가 배정된 주요 속성이 없는 경우 해당 단어도 배정하지 않고 넘어감  
        b. 해당 단어와 유사성이 높은 단어 간의 유사도가 0.8 미만인 경우에는 배정하지 않고 넘어감  
        (0.8 미만일 경우 해당 단어와 추출된 단어간의 유사 신뢰도가 낮다고 판단)
    4. 수기 라벨링을 진행하지 않은 출현 횟수 5회 미만의 속성에 대해 위의 과정을 반복  

=> 1,2번 과정은 <2.2 속성 추출 및 전처리>에서 진행함.
- [추가 설명] tokenized_data.txt 
    - 감성분석 과정에서 토큰화된 전체 리뷰를 텍스트 파일로 전환시킨 파일
    - fasttext 모델 학습에 이용
    - <추출된 속성 전처리 2.2>에서 생성된 텍스트 파일
    
- 장점 및 특징
    - 맞춤법이나 오타로 잘못 쓰여진 단어들을 원래 의미의 속성으로 분류할 수 있음
    - 명사뿐만 아니라 전치사구나 동사에도 유사한 단어를 찾고 속성을 분류할 수 있음  
    ex: pic quality -> picture quality (camera)



In [46]:
import fasttext
with tf.device(device_name):
    model = fasttext.train_unsupervised('tokenized_data.txt', epoch=10, dim=300, lr=0.1, wordNgrams=1, minCount=5)

2023-09-10 03:22:09.218669: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-10 03:22:09.218699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Read 4M words
Number of words:  10730
Number of labels: 0
Progress: 100.0% words/sec/thread:   31415 lr:  0.000000 avg.loss:  2.304548 ETA:   0h 0m 0s


In [91]:
# 처리 프로세스 참고
def similar_process(ori_aspect, pre_aspect):
    
    if pd.isnull(ori_aspect):
        return np.nan
        
    value = reverse_dic_final.get(pre_aspect)
    
    if pd.isnull(value):
        simil_aspect = model.get_nearest_neighbors(ori_aspect)[0]
        
        if simil_aspect[0] > 0.80:
            value = reverse_dic_final.get(simil_aspect[1], np.nan)
    return value

In [92]:
with tf.device(device_name):
    df_dic['simil_feature'] = np.nan
    df_dic['simil_feature'] = df_dic.apply(lambda row: similar_process(row['ori_aspect'], 
                                                                       row['pre_aspect']), axis=1)

In [93]:
df_dic = df_dic[df_dic['simil_feature']==df_dic['simil_feature']]

In [94]:
df_plus = pd.DataFrame({'aspect':['1 gb', '2 gb', '64 gb', '4 gb'], 
                        'ori_aspect':['1_gb', '2_gb', '64_gb', '4_gb'], 
                        'pre_aspect':['1_gb', '2_gb', '64_gb', '4_gb'], 
                        'count':[0,0,0,0],
                        'all_cnt':[0,0,0,0], 
                        'feature':['proc', 'proc', 'stor', 'stor'], 
                        'simil_feature':['proc', 'proc', 'stor', 'stor']})

df_dic = pd.concat([df_dic, df_plus]).reset_index(drop=True)

In [6]:
df_dic.head(3)
df_dic.shape

,aspect,ori_aspect,pre_aspect,count,all_cnt,feature,simil_feature
0,battery,battery,batteri,9048,9255,bat,bat
1,screen,screen,screen,5880,5956,dis,dis
2,battery life,battery_life,batteri_life,4111,4117,bat,bat


(1680, 7)

- **추출된 총 단어 (aspect)의 개수** = 1,680개
- **추출된 속성 정리 (열 설명)**  
    - aspect : 실제 문장 내 속성 형태
    - ori_aspect : 스테밍 없이 띄어쓰기 '_'로 대체
    - pre_aspect : 스테밍, 전치사 제거까지 수행 (속성 사전에서 분류 속성으로 사용)
    - count : aspect가 등장한 리뷰 수
    - all_cnt : 전처리된 속성 (pre_aspect)이 등장한 리뷰 수 
    - **feature** : 수기로 라벨링한 속성
    - **simil_feature** : fasttext를 통해 유사 단어로 분류된 속성 
    
*all_cnt가 따로 있는 이유 : aspect가 달라도 pre_aspect가 똑같은 경우가 존재하기 때문에

In [79]:
print('battery:', df_dic[df_dic['simil_feature']=='bat'].shape[0])
print('body:', df_dic[df_dic['simil_feature']=='body'].shape[0])
print('display:', df_dic[df_dic['simil_feature']=='dis'].shape[0])
print('camera:', df_dic[df_dic['simil_feature']=='cam'].shape[0])
print('function:', df_dic[df_dic['simil_feature']=='func'].shape[0])
print('processor:', df_dic[df_dic['simil_feature']=='proc'].shape[0])
print('storage:', df_dic[df_dic['simil_feature']=='stor'].shape[0])
print('accessory:', df_dic[df_dic['simil_feature']=='acc'].shape[0])
print('mainboard:', df_dic[df_dic['simil_feature']=='main'].shape[0])
print('')
print('최종 구축된 속성사전의 총 개수:', df_dic[df_dic['simil_feature']!='other'].shape[0])

battery: 135
body: 344
display: 166
camera: 132
function: 273
processor: 39
storage: 66
accessory: 332
mainboard: 21

최종 구축된 속성사전의 총 개수: 1508


In [60]:
import pickle
file_name = '속성사전_데이터프레임.pkl'

# 데이터를 pickle 파일로 저장
with open(file_name, 'wb') as file:
    pickle.dump(df_dic, file)

df_dic.to_csv('속성사전_데이터프레임.csv', index=False)